In [1]:
import pandas as pd
import os

Make sure to place all csv files generated by the freesurfer_stats_retrieval script in a folder. Here, the folder is called fs_stats. All csv files within that folder will be merged into one dataframe that is exported as a csv file.

In [2]:
os.chdir('fs_stats')

In [3]:
aseg_dict = {}
global_dict = {}
aparc_dict = {}

# Import csv files
for file in os.listdir():
    if file == 'asegstats_global.csv':
        global_dict[file.split('.csv')[0]] = pd.read_csv(file, delimiter = "\t")
    if (file.endswith('.csv') & file.startswith('asegstats')) & (file != 'asegstats_global.csv'):
        aseg_dict[file.split('.csv')[0]] = pd.read_csv(file, delimiter = "\t")
    if file.endswith('.csv') & (file.startswith('lh') |file.startswith('rh') ) :
        aparc_dict[file.split('.csv')[0]] = pd.read_csv(file, delimiter = "\t")

print(len(aseg_dict), len(global_dict), len(aparc_dict))

7 1 18


In [4]:
# Change column naming pattern for dataframe global
global_label_mapping = {}
global_frame = global_dict['asegstats_global']

for col in global_frame.columns[1:]:
    global_label_mapping[col] = "global_" + col.lower()
#print(global_label_mapping)

global_frame.rename(global_label_mapping, axis="columns", inplace=True)
global_dict['asegstats_global'] = global_frame

In [5]:
# Change column naming pattern for aseg dataframes
for frame in aseg_dict.values():
    label_mapping = {}

    for col in frame.columns[1:]:

        label_mapping[col] = "aseg_" + col.lower() + "_" + frame.columns[0].split(":")[1]
    
    #print(label_mapping)
    frame.rename(label_mapping, axis="columns", inplace=True)

In [6]:
# Change column naming pattern for aparc dataframes
for frame in aparc_dict.values():
    label_mapping = {}

    for col in frame.columns[1:]:
        label_mapping[col] = "aparc_" + col
    
    #print(label_mapping)
    frame.rename(label_mapping, axis="columns", inplace=True)

In [7]:
# Convert dictionaries into lists
global_list = [global_dict['asegstats_global']]
aseg_list = list(aseg_dict.values())
aparc_list = list(aparc_dict.values())

In [8]:
# Create new dataframe with id column 
df_base = pd.DataFrame()
df_base["ID"] = aseg_list[0].iloc[:,0]

# Remove 'sub' substring from id string
def remove_sub(identifier):
    id = identifier.split('-')
    return id[1].strip() 

df_base["ID"] = df_base["ID"].apply(remove_sub)
#df_base

In [9]:
# Delete ID column all other dataframes
df_list = global_list + aseg_list + aparc_list

for frame in df_list:
    frame.drop(columns=frame.columns[0], axis=1, inplace=True)

In [10]:
# Aggregate dataframes to single dataframe
joined_list = [df_base] + df_list
df = pd.concat(joined_list, axis=1)
#df

In [12]:
# Export
df.to_csv('fs_stats_combined.csv')